<a href="https://colab.research.google.com/github/GenAIHub/genai-workshop/blob/main/03_Agents/01_10k_sub_question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10K Analysis
In this demo, we explore answering complex queries by decomposing them into simpler sub-queries.

In [ ]:
!pip install boto3
!pip install llama-index
!pip install llama-index-llms-bedrock
!pip install llama-index-embeddings-bedrock

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

## Configure LLM service

In [ ]:
import os
import boto3

# Set AWS env config
region_name = os.getenv("AWS_REGION", "us-east-1")
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID", "")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY", "")

In [ ]:
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.core import Settings

llm_model_id = "anthropic.claude-3-haiku-20240307-v1:0"
embed_model_id = "amazon.titan-embed-text-v1"

llm = Bedrock(
    model=llm_model_id,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name,
    temperature=0.1,
    max_tokens=512
    )

embed_model = BedrockEmbedding(
    model=embed_model_id,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name,
    )

Settings.llm = llm
Settings.embed_model = embed_model

## Download Data

In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

## Load data

In [ ]:
lyft_docs = SimpleDirectoryReader(
    input_files=["./data/10k/lyft_2021.pdf"]
).load_data()
uber_docs = SimpleDirectoryReader(
    input_files=["./data/10k/uber_2021.pdf"]
).load_data()

## Build indices

In [ ]:
lyft_index = VectorStoreIndex.from_documents(lyft_docs)

In [ ]:
uber_index = VectorStoreIndex.from_documents(uber_docs)

## Build query engines

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)

In [ ]:
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021"
            ),
        ),
    ),
]

s_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=False
)

## Run queries

In [ ]:
response = s_engine.query(
    "Compare and contrast the customer segments and geographies that grew the"
    " fastest"
)

In [ ]:
print(response)

In [ ]:
response = s_engine.query(
    "Compare revenue growth of Uber and Lyft from 2020 to 2021"
)

In [ ]:
print(response)